# 概要

ニコニコでコンバージョンが得られやすいジャンルを探す

## コード

In [1]:
ENV["GKS_ENCODING"] = "utf-8"
using HTTP, JSON, CSV, TableOperations, DataFrames, LightXML, Plots, Measures

"""
HTTPでデータを取得する関数
"""
function api_get(url)
  count = 0
  while count < 10
    try
      response = HTTP.get(url)
      response.status == 200 && (
        return String(response.body);
      )
    finally
      count += 1
    end
  end
  return error("Error: bad url")
end

"""
ニコニコ動画のデータを収集しファイルに保存する関数

参考: https://dwango.github.io/niconico/genre_ranking/ranking_log
"""
function collect_nico_data(ranking_type, date, file_type, endpoint="https://dcdn.cdn.nimg.jp/nicovideo/old-ranking")
  url       = "$(endpoint)/$(ranking_type)/$(date)/$(file_type)"
  file_name = replace(url, "/" => "_")
  write(file_name, api_get(url))
  return file_name
end

"""
ファイル名リストからタグを検索し、ファイル名を返す関数
"""
function search_tag(tag, file_name)
  file_data_list = JSON.parsefile(file_name)
  for file_data ∈ file_data_list
    file_data["tag"] == tag && (
      return file_data["file"]
    )
  end
  return error("Error: $(tag) is not found")
end

"""
ファイル名からデータを集計し、タグごとのデータを返す関数
"""
function aggregate_by_tag(ac6_result_file)

  function get_tags_by_id(id)
    xml_element = "https://ext.nicovideo.jp/api/getthumbinfo/$(id)" |> api_get |> parse_string |> root
    for class ∈ ["thumb", "tags"]
      xml_element = find_element(xml_element, class)
      isnothing(xml_element) && (
        #=
        ここに入るときは削除されてるかも
        <nicovideo_thumb_response status="fail">
          <error>
            <code>DELETED</code>
            <description>deleted</description>
          </error>
        </nicovideo_thumb_response>
        =#
        return [];
      )
    end
    elements = [content(e) for e ∈ get_elements_by_tagname(xml_element, "tag")]
  end

  tag_data         = Dict{String,Dict{String,Int64}}()
  tag_data_by_view = Dict{String,Dict{String,Float64}}()
  ac6_results      = JSON.parsefile(ac6_result_file)
  AGGREGATE_COLS   = ["view", "like", "comment", "mylist"]
  tag_unique_set   = Set{String}()
  for ac6_result ∈ ac6_results
    tags = get_tags_by_id(ac6_result["id"])
    for tag ∈ tags
      push!(tag_unique_set, tag)
      haskey(tag_data, tag) || (
        tag_data[tag]         = Dict(col => 0 for col ∈ AGGREGATE_COLS);
        tag_data_by_view[tag] = Dict(col => 0 for col ∈ AGGREGATE_COLS);
      )
      for col ∈ AGGREGATE_COLS
        tag_data[tag][col] += ac6_result["count"][col]
      end
    end
  end
  for tag ∈ tag_unique_set
    for col ∈ ["like", "comment", "mylist"]
      tag_data_by_view[tag][col] = tag_data[tag][col] * 1000 / tag_data[tag]["view"]
    end
  end
  write("aggregate_$(ac6_result_file)", JSON.json(tag_data))
  write("aggregate_by_view_$(ac6_result_file)", JSON.json(tag_data_by_view))
  return tag_data_by_view
end

"""
視聴回数あたりのいいね数、コメント数、マイリスト数を計算し、視覚化する
data_type: "julia_dict" or "json_file"
"""
function plot_data_by_view(tag_data_by_view, data_type = "julia_dict")

  data_type == "json_file" && (
    cd(@__DIR__());
    tag_data_by_view = JSON.parsefile(tag_data_by_view);
  )

  gr(fontfamily="JuliaMono")
  # タイトルとリアクション割合の棒グラフを作成する
  df         = DataFrame()
  df.tag     = collect(keys(tag_data_by_view))
  df.view    = [tag_data_by_view[t]["view"]    for t in df.tag]
  df.mylist  = [tag_data_by_view[t]["mylist"]  for t in df.tag]
  df.like    = [tag_data_by_view[t]["like"]    for t in df.tag]
  df.comment = [tag_data_by_view[t]["comment"] for t in df.tag]
  df         = sort(df, :view, rev=false)
  df         = df[end-41:end-1, :]

  # 棒グラフを作成する
  my_bar(x,y,str,n) = bar(
    x,
    y,
    fontfamily  = "JuliaMono",
    orientation = "horizontal",
    yticks      = (0.5:length(x) , x .|> (t -> t * " "^(13) )),
    xticks      = (0:n:maximum(y), string.(Int.(collect(0:n:maximum(y))))),
    title       = "\nアーマードコア6 タグ毎の視聴回数1000回あたりの$(str)数                              ",
    size        = (2000, 1350),
    bar_width   = 0.7,
    left_margin = 10mm,
    right_margin= 5mm,
    widen       = true,
    legend      = false,
    ylimit      = (0, length(df.tag) + 1),
  )
  
  plot(
  my_bar(df.tag, df.mylist, "マイリスト", 5),
  # savefig("ac6_mylist_per_view.svg")
  my_bar(df.tag, df.like, "いいね", 10),
  # savefig("ac6_like_per_view.svg")
  my_bar(df.tag, df.comment, "コメント", 20),
  # savefig("ac6_comment_per_view.svg")
  )
  savefig("ac6_reaction_per_view.svg")
end
file_name_list_name = collect_nico_data("daily", "2023-09-23", "file_name_list.json")
ac6_file_name       = search_tag("アーマードコア6", file_name_list_name)
ac6_result_file     = collect_nico_data("daily", "2023-09-23", ac6_file_name)
tag_data_by_view    = aggregate_by_tag(ac6_result_file)
plot_data_by_view("aggregate_by_view_https:__dcdn.cdn.nimg.jp_nicovideo_old-ranking_daily_2023-09-23_game_04.json", "json_file")

plot_data_by_view

![結果](./ac6_reaction_per_view.svg)